## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [2]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [3]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: f28c564a95cc4be79aa43ca5d18d1219>,[DO1 data],"Dataset of 6 samples, 1 feature",<class 'torch.Tensor'>


In [4]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: e6acf34195bc4f32b96e623ded801bf7>,[DO2 data],"Dataset of 5 samples, 1 feature",<class 'torch.Tensor'>


## Linear regression

In [5]:
data1_ptr = duet1.store[0]
data2_ptr = duet2.store[0]

print(data1_ptr)
print(data2_ptr)

### Create Base Model

In [6]:
import torch

In [7]:
in_dim = 1
out_dim = 1

In [8]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

### Training

In [9]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        loss = torch_ref.nn.functional.mse_loss(output, target_ptr)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses

#### Send one copy of the model to each data owner or client and train them remotely one by one

In [10]:
import torch as th
import numpy as np

In [11]:
base_model = SyNet(torch)

Train on Data Owner 1 data

In [12]:
remote_model1 = base_model.send(duet1)

In [13]:
remote_torch1 = duet1.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

Dummy target data

In [14]:
target1_ptr = th.FloatTensor(np.array([5, 10, 15, 22, 30, 38]).reshape(-1, 1))
target1_ptr

tensor([[ 5.],
        [10.],
        [15.],
        [22.],
        [30.],
        [38.]])

In [15]:
iteration = 10
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

Epoch 0 loss 881.3590698242188
Epoch 1 loss 683.983154296875
Epoch 2 loss 512.5094604492188
Epoch 3 loss 367.18212890625
Epoch 4 loss 247.9085235595703
Epoch 5 loss 154.1551971435547
Epoch 6 loss 84.83381652832031
Epoch 7 loss 38.19247817993164
Epoch 8 loss 11.740914344787598
Epoch 9 loss 2.2487456798553467


Train on Data Owner 2 data

In [16]:
remote_model2 = base_model.send(duet2)

In [17]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

Dummy Target data

In [18]:
target2_ptr = th.FloatTensor(np.array([35, 40, 45, 55, 60]).reshape(-1, 1))
target2_ptr

tensor([[35.],
        [40.],
        [45.],
        [55.],
        [60.]])

In [19]:
iteration = 10
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 4037.137939453125
Epoch 1 loss 3106.560546875
Epoch 2 loss 2301.989501953125
Epoch 3 loss 1624.5501708984375
Epoch 4 loss 1073.6298828125
Epoch 5 loss 646.3277587890625
Epoch 6 loss 336.84844970703125
Epoch 7 loss 135.93789672851562
Epoch 8 loss 30.522693634033203
Epoch 9 loss 3.782020092010498


### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [20]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[-0.2126]], requires_grad=True), Parameter containing:
tensor([0.7599], requires_grad=True)]

Remote model1 parameters:
[Parameter containing:
tensor([[0.6804]], requires_grad=True), Parameter containing:
tensor([1.6496], requires_grad=True)]

Remote model2 parameters:
[Parameter containing:
tensor([[0.6740]], requires_grad=True), Parameter containing:
tensor([1.6455], requires_grad=True)]


As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [21]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

OrderedDict([('linear.weight', tensor([[0.6804]])), ('linear.bias', tensor([1.6496]))])


In [22]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('linear.weight', tensor([[0.6740]])), ('linear.bias', tensor([1.6455]))])


In [23]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [24]:
avg_updates = OrderedDict()
avg_updates["linear.weight"] = (
    remote_model1_updates["linear.weight"] + remote_model2_updates["linear.weight"]
) / 2
avg_updates["linear.bias"] = (
    remote_model1_updates["linear.bias"] + remote_model2_updates["linear.bias"]
) / 2

print(avg_updates)

OrderedDict([('linear.weight', tensor([[0.6772]])), ('linear.bias', tensor([1.6475]))])


### Load aggregated weights

In [25]:
combined_model = SyNet(torch)

In [26]:
combined_model.load_state_dict(avg_updates)

In [27]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [28]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [29]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 13.160049438476562 Ground Truth: 12.0
Prediction: 18.577699661254883 Ground Truth: 15.0
Prediction: 23.31814193725586 Ground Truth: 20.0
Prediction: 35.50785446166992 Ground Truth: 30.0
Prediction: 55.82403564453125 Ground Truth: 50.0


In [30]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: -2.854454517364502 Ground Truth: 12.0
Prediction: -4.55531644821167 Ground Truth: 15.0
Prediction: -6.043570518493652 Ground Truth: 20.0
Prediction: -9.87051010131836 Ground Truth: 30.0
Prediction: -16.248741149902344 Ground Truth: 50.0


## Comparison to classical linear regression on centralised data

In [31]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [32]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [33]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [34]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [35]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

Epoch 0 loss 40.29050827026367
Epoch 10 loss 5.373466968536377
Epoch 20 loss 6.004182815551758
Epoch 30 loss 3.594534158706665
Epoch 40 loss 3.0856475830078125
Epoch 50 loss 3.0363523960113525
Epoch 60 loss 2.998192548751831
Epoch 70 loss 2.970550537109375
Epoch 80 loss 2.953270196914673
Epoch 90 loss 2.9392645359039307


In [36]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [37]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 11.218692779541016 Ground Truth: 12.0
Prediction: 16.17717933654785 Ground Truth: 15.0
Prediction: 20.515853881835938 Ground Truth: 20.0
Prediction: 31.672447204589844 Ground Truth: 30.0
Prediction: 50.26677322387695 Ground Truth: 50.0
